In [4]:
!pip install tensorflow_datasets
!pip install tensorflow_recommenders

     |████████████████████████████████| 85 kB 2.9 MB/s 
     |████████████████████████████████| 462 kB 44.5 MB/s 


In [6]:
from typing import Dict, Text

import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

# Ratings data.
ratings = tfds.load('movielens/100k-ratings', split="train")
# Features of all the available movies.
movies = tfds.load('movielens/100k-movies', split="train")

# Select the basic features.
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"]
})
movies = movies.map(lambda x: x["movie_title"])


user_ids_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(ratings.map(lambda x: x["user_id"]))

movie_titles_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
movie_titles_vocabulary.adapt(movies)

class MovieLensModel(tfrs.Model):
  # We derive from a custom base class to help reduce boilerplate. Under the hood,
  # these are still plain Keras Models.

  def __init__(
      self,
      user_model: tf.keras.Model,
      movie_model: tf.keras.Model,
      task: tfrs.tasks.Retrieval):
    super().__init__()

    # Set up user and movie representations.
    self.user_model = user_model
    self.movie_model = movie_model

    # Set up a retrieval task.
    self.task = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # Define how the loss is computed.

    user_embeddings = self.user_model(features["user_id"])
    movie_embeddings = self.movie_model(features["movie_title"])

    return self.task(user_embeddings, movie_embeddings)


# Define user and movie models.
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocab_size(), 64)
])
movie_model = tf.keras.Sequential([
    movie_titles_vocabulary,
    tf.keras.layers.Embedding(movie_titles_vocabulary.vocab_size(), 64)
])

# Define your objectives.
task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    movies.batch(128).map(movie_model)
  )
)



# Create a retrieval model.
model = MovieLensModel(user_model, movie_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

# Train for 3 epochs.
model.fit(ratings.batch(4096), epochs=1)

# Use brute-force search to set up retrieval using the trained representations.
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
    movies.batch(100).map(lambda title: (title, model.movie_model(title))))






25/25 [==============================] - 30s 1s/step - factorized_top_k/top_1_categorical_accuracy: 3.0000e-05 - factorized_top_k/top_5_categorical_accuracy: 0.0014 - factorized_top_k/top_10_categorical_accuracy: 0.0043 - factorized_top_k/top_50_categorical_accuracy: 0.0439 - factorized_top_k/top_100_categorical_accuracy: 0.1012 - loss: 33090.9091 - regularization_loss: 0.0000e+00 - total_loss: 33090.9091


ValueError: ignored

NotImplementedError: ignored

In [7]:

# Get some recommendations.
_, titles = index(np.array(["42"]))
print(f"Top 3 recommendations for user 42: {titles[0, :3]}")



Top 3 recommendations for user 42: [b'Rent-a-Kid (1995)'
 b'Far From Home: The Adventures of Yellow Dog (1995)'
 b'Winnie the Pooh and the Blustery Day (1968)']


In [ ]:
model.compute_output_shape(input_shape=(None, 256, 256, 3))

In [11]:
model.save('tf_save/tf_model')
index.save('tf_save/index')

ValueError: ignored

In [9]:
import pickle

In [10]:
pickle.dump(model, open('model.pkl', 'wb'))

INFO:tensorflow:Assets written to: ram://c7788dbd-a4e7-4716-94ae-c83c8d8d3ae9/assets


INFO:tensorflow:Assets written to: ram://c7788dbd-a4e7-4716-94ae-c83c8d8d3ae9/assets


INFO:tensorflow:Assets written to: ram://8a42583b-a8c2-4d17-a3b3-3537129b5d76/assets


INFO:tensorflow:Assets written to: ram://8a42583b-a8c2-4d17-a3b3-3537129b5d76/assets


TypeError: ignored

In [19]:
_, titles = index({"user_id": np.array(["32"]) ,"user_occupation_label": np.array([20])})


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:559: UserWarning: Input dict contained keys ['user_id', 'user_occupation_label'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


In [20]:

index.save('./index_model')

INFO:tensorflow:Assets written to: ./index_model/assets


INFO:tensorflow:Assets written to: ./index_model/assets


In [21]:
new_model = tf.keras.models.load_model('./index_model')

In [25]:
# Get some recommendations.
_, new_titles = index(np.array(["42"]))
print(f"Top recommendations for user 42: {new_titles}")


Top recommendations for user 42: [[b'Rent-a-Kid (1995)'
  b'Far From Home: The Adventures of Yellow Dog (1995)'
  b'Winnie the Pooh and the Blustery Day (1968)'
  b'Aristocats, The (1970)' b'Aladdin and the King of Thieves (1996)'
  b'House Arrest (1996)' b'First Knight (1995)' b'Pinocchio (1940)'
  b'Old Yeller (1957)' b'Andre (1994)']]


In [28]:
!zip index_model.zip index_model

  adding: index_model/ (stored 0%)
